In [1]:
# DuckDB api

In [2]:
import duckdb
conn = duckdb.connect('my_fico_data.duckdb')

conn.execute("""
    CREATE TABLE IF NOT EXISTS fico_scores (
        person_number TEXT,
        fico_score INT,
        score_date DATE         
    )
""")

conn.execute("INSERT INTO fico_scores VALUES ('P123', 710, '2025-06-30')")

In [3]:
results = conn.execute("SELECT * FROM fico_scores").fetchall()
print(results)

[('P123', 710, datetime.date(2025, 6, 30))]


In [4]:
conn.close()

In [8]:
from pathlib import Path
PERMISSIONS_DB = Path(r"Z:\Credit_Loan_Review\Alerts\Production\Xactus\Permission_db\pfs_permission.csv")

conn = duckdb.connect()
results = conn.execute("""
    SELECT
        *
    FROM
        read_csv(?, header=false)
    WHERE
        column0 = 327
""", [str(PERMISSIONS_DB)]).fetchall()

print(results)


ConversionException: Conversion Error: Could not convert string 'persnbr' to INT32 when casting from source column column0

LINE 7:         column0 = 327
                ^

In [ ]:
""" 
Alerts: Main entry point
Developed by CD


Usage:
    python -m src.main
"""

from io import StringIO
import time
import os
from datetime import datetime, timedelta, date
from typing import List
from collections import defaultdict, Counter

from io import StringIO
from pathlib import Path
import asyncio
import sys

import pandas as pd # type: ignore
import numpy as np # type: ignore

import src.cdutils.caching
import src.cdutils.input_cleansing
import src.cdutils.database
import src.control_panel
import src.flags
import src.flags.credit_score
import src.flags.deposits
import src.transformation.joining
import src.transformation.total_exposure
import src.transformation.cleaning
import src.transformation.filtering
import src.household
import src.personal_guarantor
import src.flags.past_due
import src.flags.line_utilization
from src._version import __version__


# def main(production_flag: bool=False):
#     if production_flag:
#         BASE_PATH = Path(r'\\00-DA1\Home\Share\Line of Business_Shared Services\Credit_Loan_Review\Alerts\Production')
#         assert "prod" in __version__, (f"Cannot run in production mode without 'prod' in the __version__")
#     else:
#         BASE_PATH = Path('.')

    # Database Connection Configuration
data = src.cdutils.database.fetch_data()

# Unpack
acctcommon = data['acctcommon'].copy()
acctloan = data['acctloan'].copy()
loans = data['loans'].copy()
househldacct = data['househldacct'].copy()
acctstatistichist = data['acctstatistichist'].copy()
acctloanlimithist = data['acctloanlimithist'].copy()
allroles = data['allroles'].copy()
pers = data['pers'].copy()
viewperstaxid = data['viewperstaxid'].copy()

# Input cleaning
viewperstaxid_schema = {
'persnbr': int,
'taxid': float
}

viewperstaxid = src.cdutils.input_cleansing.enforce_schema(viewperstaxid, viewperstaxid_schema)

# # Core ETL
loan_data = src.transformation.joining.filter_and_merge_loan_tables(acctcommon, acctloan, loans)
loan_data = src.transformation.total_exposure.append_total_exposure_field(loan_data)
househldacct = src.transformation.cleaning.drop_hh_duplicates(househldacct)
loan_data = src.transformation.joining.append_household_number(loan_data, househldacct)
household_grouping_df = src.household.household_total_exposure(loan_data)
loan_data = src.household.append_household_exposure(loan_data, household_grouping_df)

# Main parameters for loans
loan_data = src.transformation.filtering.filter_to_target_products(
    loan_data,
    creditlimitamt=500000,
    total_exposure=1000000
) # Note that minor list is hard-coded to line of credit minors in the filtering module

# Additional ETL
acctstatistic_output = src.transformation.cleaning.acctstatistichist_cleaning(acctstatistichist, acctcommon)
pd_df = src.flags.past_due.count_pd(acctstatistic_output)
pd30_df = src.flags.past_due.count_pd30(acctstatistic_output)
loan_data = src.flags.past_due.append_pd_info(loan_data, pd_df, pd30_df)
deposit_data = src.flags.deposits.deposit_criteria_testing(househldacct)
loan_data = src.flags.deposits.append_deposit_data(loan_data, deposit_data)
utilization_data, cleanup_data = src.flags.line_utilization.line_utilization_fetch(loan_data)
loan_data = src.flags.line_utilization.append_line_utilization_data(loan_data, utilization_data, cleanup_data)
inactive_date_df = src.transformation.joining.get_inactive_date(acctloanlimithist)
loan_data = src.transformation.joining.append_inactive_date(loan_data, inactive_date_df)



In [ ]:
loan_data

In [2]:
pers_data = src.personal_guarantor.append_tax_id_to_pers(pers, viewperstaxid)
pers_data = src.personal_guarantor.append_credit_score(pers_data)


In [ ]:
pers_data

In [ ]:
allroles

In [4]:

credit_score_data = src.personal_guarantor.allroles_with_credit_score(allroles, pers_data)

# # Control panel for additional parameters
# loan_data = src.control_panel.criteria_flags(
#     loan_data,
#     credit_score_data,
#     score_floor = 680, # Minimum Credit Score for any guarantor related to the loan
#     score_decrease = -0.1, # Decrease of credit score since prior period
#     ttm_pd_amt = 3, # Times in trailing 12 Months Past Due (15-29)
#     ttm_pd30_amt = 1, # Times in trailing 12 Months Past Due (30+)
#     ttm_overdrafts = 5, # Number of days with an overdrawn balance in trailing 12 Months
#     deposit_change_pct = -.3, # Aggregate deposits in relationship decrease (-30% loss)
#     min_deposits = 50000, # Minimum deposits to be eligible to flag deposit decrease
#     utilization_limit = .7 # Line Utilization flag for trailing 12 months
# )

# # Consolidation of the columns necessary
# final_df = loan_data[['acctnbr','effdate','ownername','product','loanofficer','inactivedate','Net Balance','Net Available','Net Collateral Reserve','cobal','creditlimitamt','Total Exposure_hhgroup','ttm_pd','ttm_pd30','TTM Days Overdrawn','3Mo_AvgBal','TTM_AvgBal','Deposit Change Pct','ttm line utilization','cleanup_provision','riskratingcd','past_due_flag','ttm_overdrafts_flag','deposit_change_flag','ttm_utilization_flag','score_flag','passed_all_flag']]

# # Writing output
# ALERTS_PATH = BASE_PATH / Path('./Output/alerts.xlsx')
# final_df.to_excel(ALERTS_PATH, index=False, engine='openpyxl', sheet_name='Sheet1')

# CREDSCORE_PATH = BASE_PATH / Path('./Output/credit_score_detail.xlsx')
# credit_score_data.to_excel(CREDSCORE_PATH, index=False, engine='openpyxl', sheet_name='Sheet1')
    
# print('Execution Complete!')

# if __name__ == "__main__":
# print(f"Starting alerts [{__version__}]")
# # main(production_flag=True)
# main()
# print("Complete!")








In [ ]:
credit_score_data

In [ ]:
### Guarantor file creation
""" 
Alerts: Main entry point
Developed by CD


Usage:
    python -m src.main
"""

from pathlib import Path
import os

import pandas as pd # type: ignore
import numpy as np # type: ignore

import cdutils.input_cleansing # type: ignore
import src.fetch_data
import src.control_panel
import src.flags
import src.flags.credit_score
import src.flags.deposits
import cdutils.joining # type: ignore
import cdutils.loans.calculations # type: ignore
import src.transformation.cleaning
import src.transformation.filtering
import src.transformation.total_exposure
import src.transformation.joining
import src.household
import src.personal_guarantor
import src.flags.past_due
import src.flags.line_utilization
from src._version import __version__


# def main(production_flag: bool=False):
#     if production_flag:
#         BASE_PATH = Path(r'\\00-DA1\Home\Share\Line of Business_Shared Services\Credit_Loan_Review\Alerts\Production')
#         assert "prod" in __version__, (f"Cannot run in production mode without 'prod' in the __version__")
#     else:
#         BASE_PATH = Path('.')

# Database Connection Configuration
data = src.fetch_data.fetch_data()

# Unpack
acctcommon = data['acctcommon'].copy()
acctloan = data['acctloan'].copy()
loans = data['loans'].copy()
househldacct = data['househldacct'].copy()
acctstatistichist = data['acctstatistichist'].copy()
acctloanlimithist = data['acctloanlimithist'].copy()
allroles = data['allroles'].copy()
pers = data['pers'].copy()
viewperstaxid = data['viewperstaxid'].copy()

# Input cleaning
viewperstaxid_schema = {
'persnbr': int,
'taxid': float
}

viewperstaxid = cdutils.input_cleansing.enforce_schema(viewperstaxid, viewperstaxid_schema)


AttributeError: module 'cdutils.joining' has no attribute 'filter_and_merge_loan_tables'

In [ ]:
# # Core ETL

loan_data = src.transformation.joining.filter_and_merge_loan_tables(acctcommon, acctloan, loans)
loan_data = src.transformation.total_exposure.append_total_exposure_field(loan_data)
househldacct = src.transformation.cleaning.drop_hh_duplicates(househldacct)
loan_data = src.transformation.joining.append_household_number(loan_data, househldacct)
household_grouping_df = src.household.household_total_exposure(loan_data)
loan_data = src.household.append_household_exposure(loan_data, household_grouping_df)

# Main parameters for loans
loan_data = src.transformation.filtering.filter_to_target_products(
    loan_data,
    creditlimitamt=500000,
    total_exposure=1000000
) # Note that minor list is hard-coded to line of credit minors in the filtering module

# Additional ETL
acctstatistic_output = src.transformation.cleaning.acctstatistichist_cleaning(acctstatistichist, acctcommon)
pd_df = src.flags.past_due.count_pd(acctstatistic_output)
pd30_df = src.flags.past_due.count_pd30(acctstatistic_output)
loan_data = src.flags.past_due.append_pd_info(loan_data, pd_df, pd30_df)
deposit_data = src.flags.deposits.deposit_criteria_testing()
loan_data = src.flags.deposits.append_deposit_data(loan_data, deposit_data)
utilization_data, cleanup_data = src.flags.line_utilization.line_utilization_fetch(loan_data)
loan_data = src.flags.line_utilization.append_line_utilization_data(loan_data, utilization_data, cleanup_data)
inactive_date_df = src.transformation.joining.get_inactive_date(acctloanlimithist)
loan_data = src.transformation.joining.append_inactive_date(loan_data, inactive_date_df)


In [15]:
pers_data = src.personal_guarantor.append_tax_id_to_pers(pers, viewperstaxid)


In [ ]:
pers_data = src.personal_guarantor.append_credit_score(pers_data)
credit_score_data = src.personal_guarantor.allroles_with_credit_score(allroles, pers_data)



In [8]:
loan_data

,acctnbr,effdate,mjaccttypcd,product,currmiaccttypcd,bookbalance,loanofficer,ownername,curracctstatcd,contractdate,...,ytd_pd30,ttm_pd30,3Mo_AvgBal,TTM_AvgBal,TTM Days Overdrawn,TTM NSF,Deposit Change Pct,ttm line utilization,cleanup_provision,inactivedate
0,600086981811,2025-06-09,CML,Line of Credit,CM30,13000,SBLC LOAN OFFICER,IPSCO INC,ACT,2017-07-27,...,0.0,0.0,7.223633e+04,1.034494e+05,0.0,0.0,-0.301723,0.167680,0,2025-09-30
1,300222581209,2025-06-09,CML,Line of Credit,CM30,0,SBLC LOAN OFFICER,OTERI FUNERAL HOME INC,ACT,2009-09-04,...,0.0,0.0,1.887642e+05,1.911391e+05,0.0,0.0,-0.012425,0.000000,0,2026-06-30
2,150474030,2025-06-09,CML,Line of Credit,CM30,0,SBLC LOAN OFFICER,"COOGAN SMITH, LLP",ACT,2020-06-12,...,0.0,0.0,2.304757e+05,2.042852e+05,4.0,0.0,0.128205,0.000000,0,2025-08-31
3,150304683,2025-06-09,CML,Line of Credit,CM30,100000,THOMAS D. KELLY,2120 PLEASANT STREET REALTY TRUST,ACT,2019-04-05,...,0.0,0.0,3.026390e+03,3.357974e+03,1.0,1.0,-0.098745,1.000000,1,2025-11-30
4,151160167,2025-06-09,CML,Express Business LOC,CM57,0,EBL PROGRAM ADMIN,"TAUNTON ANTIQUES CENTER, INC.",ACT,2025-04-18,...,0.0,0.0,2.706915e+04,4.860734e+04,0.0,5.0,-0.443106,0.000000,0,2026-04-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,151189373,2025-06-09,CML,Express Business LOC,CM57,40136.97,EBL PROGRAM ADMIN,"NEWBURY DESIGN ASSOCIATES, INC.",ACT,2025-05-30,...,0.0,0.0,4.513469e+04,8.051920e+04,0.0,0.0,-0.439454,0.271196,1,2026-05-30
283,600143780699,2025-06-09,CML,Borrowing Base Line of Credit,CM06,0,ALISSA E. HALL,BI COUNTY COLLABORATIVE,ACT,2003-04-17,...,0.0,0.0,9.027144e+05,8.759239e+05,0.0,0.0,0.030585,0.000000,0,2025-12-31
284,400261381666,2025-06-09,CML,Equipment Line of Credit,CM11,0,MARK A. BORKMAN,YANKEE LEASING CORPORATION,ACT,2015-05-21,...,0.0,0.0,1.004472e+06,1.066577e+06,0.0,0.0,-0.058228,0.000000,0,2025-07-31
285,150642942,2025-06-09,CML,Line of Credit,CM30,87141.56,SBLC LOAN OFFICER,"M.O.L.I.F.E., INC.",ACT,2021-08-18,...,0.0,0.0,1.241809e+05,1.274998e+05,0.0,0.0,-0.026031,0.946536,1,2025-12-31


In [ ]:
allroles

In [13]:
eligible_guar = pd.merge(loan_data, allroles, on='acctnbr',how='left')

In [14]:
eligible_guar

,acctnbr,effdate,mjaccttypcd,product,currmiaccttypcd,bookbalance,loanofficer,ownername,curracctstatcd,contractdate,...,ttm line utilization,cleanup_provision,inactivedate,acctrolecd,acctroledesc,emplroleyn,persnbr,orgnbr,rundate,datelastmaint
0,600086981811,2025-06-09,CML,Line of Credit,CM30,13000,SBLC LOAN OFFICER,IPSCO INC,ACT,2017-07-27,...,0.167680,0,2025-09-30,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,300222581209,2025-06-09,CML,Line of Credit,CM30,0,SBLC LOAN OFFICER,OTERI FUNERAL HOME INC,ACT,2009-09-04,...,0.000000,0,2026-06-30,GUAR,Guarantor,N,1029020.0,NaN,2025-06-09,2025-06-09 21:22:55
2,150474030,2025-06-09,CML,Line of Credit,CM30,0,SBLC LOAN OFFICER,"COOGAN SMITH, LLP",ACT,2020-06-12,...,0.000000,0,2025-08-31,NaN,NaN,NaN,NaN,NaN,NaT,NaT
3,150304683,2025-06-09,CML,Line of Credit,CM30,100000,THOMAS D. KELLY,2120 PLEASANT STREET REALTY TRUST,ACT,2019-04-05,...,1.000000,1,2025-11-30,GUAR,Guarantor,N,421.0,NaN,2025-06-09,2025-06-09 21:22:54
4,150304683,2025-06-09,CML,Line of Credit,CM30,100000,THOMAS D. KELLY,2120 PLEASANT STREET REALTY TRUST,ACT,2019-04-05,...,1.000000,1,2025-11-30,GUAR,Guarantor,N,1027703.0,NaN,2025-06-09,2025-06-09 21:22:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,600143780699,2025-06-09,CML,Borrowing Base Line of Credit,CM06,0,ALISSA E. HALL,BI COUNTY COLLABORATIVE,ACT,2003-04-17,...,0.000000,0,2025-12-31,NaN,NaN,NaN,NaN,NaN,NaT,NaT
397,400261381666,2025-06-09,CML,Equipment Line of Credit,CM11,0,MARK A. BORKMAN,YANKEE LEASING CORPORATION,ACT,2015-05-21,...,0.000000,0,2025-07-31,GUAR,Guarantor,N,1067999.0,NaN,2025-06-09,2025-06-09 21:22:55
398,400261381666,2025-06-09,CML,Equipment Line of Credit,CM11,0,MARK A. BORKMAN,YANKEE LEASING CORPORATION,ACT,2015-05-21,...,0.000000,0,2025-07-31,GUAR,Guarantor,N,NaN,1001562.0,2025-06-09,2025-06-09 21:22:57
399,150642942,2025-06-09,CML,Line of Credit,CM30,87141.56,SBLC LOAN OFFICER,"M.O.L.I.F.E., INC.",ACT,2021-08-18,...,0.946536,1,2025-12-31,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [ ]:
PERMISSIONS_INPUT = Path(r"Z:\Credit_Loan_Review\Alerts\Production\assets\xactus\pfs_permission.csv")


import os
def permission_layer(df, permissions_csv_path):
    """
    Filter dataframe based on permissions and return filtered data plus exceptions.
    
    Args:
        df (pd.DataFrame): Input dataframe containing persnbr column
        permissions_csv_path (str): Absolute path to permissions CSV file
    
    Returns:
        tuple: (filtered_df, exceptions_df)
            - filtered_df: Records with valid permissions (inner join result)
            - exceptions_df: Records from input df without permissions
    
    Raises:
        FileNotFoundError: If permissions CSV doesn't exist
        KeyError: If persnbr column missing from either dataframe
        AssertionError: If duplicate persnbr found in either dataframe
        ValueError: If either dataframe is empty after preprocessing
    """
    
    # Validate permissions file exists
    if not os.path.exists(permissions_csv_path):
        raise FileNotFoundError(f"Permissions file not found: {permissions_csv_path}")
    
    # Read permissions CSV
    permissions_df = pd.read_csv(permissions_csv_path)
    
    # Validate required columns exist
    if 'persnbr' not in df.columns:
        raise KeyError("Column 'persnbr' not found in input dataframe")
    
    if 'persnbr' not in permissions_df.columns:
        raise KeyError("Column 'persnbr' not found in permissions CSV")
    
    # Drop rows where persnbr is null from input df
    df_clean = df.dropna(subset=['persnbr']).copy()
    
    if df_clean.empty:
        raise ValueError("Input dataframe is empty after removing null persnbr values")
    
    if permissions_df.empty:
        raise ValueError("Permissions dataframe is empty")
    
    # Check for duplicate persnbr in both dataframes
    if df_clean['persnbr'].duplicated().any():
        duplicates = df_clean[df_clean['persnbr'].duplicated(keep=False)]['persnbr'].unique()
        raise AssertionError(f"Duplicate persnbr found in input dataframe: {duplicates}")
    
    if permissions_df['persnbr'].duplicated().any():
        duplicates = permissions_df[permissions_df['persnbr'].duplicated(keep=False)]['persnbr'].unique()
        raise AssertionError(f"Duplicate persnbr found in permissions dataframe: {duplicates}")
    
    # Perform outer merge to identify all cases
    merged_df = df_clean.merge(
        permissions_df, 
        on='persnbr', 
        how='outer', 
        indicator=True,
        suffixes=('', '_perm')
    )
    
    # Filter for records that exist in both (inner join result)
    filtered_df = merged_df[merged_df['_merge'] == 'both'].drop('_merge', axis=1)
    
    # Identify exceptions: records in input df but not in permissions
    exceptions_df = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis=1)
    
    # Clean up exceptions_df by removing permission columns (they'll be NaN anyway)
    perm_cols = [col for col in exceptions_df.columns if col.endswith('_perm')]
    exceptions_df = exceptions_df.drop(perm_cols, axis=1)
    
    return filtered_df, exceptions_df


In [16]:
pers_data

,persnbr,firstname,lastname,datebirth,deathnotificationdate,taxid
0,127,DEPOSIT8,COCC,NaT,NaT,NaN
1,180,ROCH2,COCC,NaT,NaT,NaN
2,181,ROCH3,COCC,NaT,NaT,NaN
3,182,ROCH4,COCC,NaT,NaT,NaN
4,184,PRINT6,COCC,NaT,NaT,NaN
...,...,...,...,...,...,...
166491,1174028,KENNETH,PEREIRA,1989-09-03,NaT,31745726.0
166492,1174035,JOSEPH,DONNELLY,1951-05-10,NaT,19420302.0
166493,1174042,CARLOS,FURTADO,1973-01-29,NaT,11740073.0
166494,1174098,HOLLY,PILOTTE,1974-09-23,NaT,12666119.0


In [17]:
pers

,persnbr,firstname,lastname,datebirth,deathnotificationdate
0,127,DEPOSIT8,COCC,NaT,NaT
1,180,ROCH2,COCC,NaT,NaT
2,181,ROCH3,COCC,NaT,NaT
3,182,ROCH4,COCC,NaT,NaT
4,184,PRINT6,COCC,NaT,NaT
...,...,...,...,...,...
166491,1174028,KENNETH,PEREIRA,1989-09-03,NaT
166492,1174035,JOSEPH,DONNELLY,1951-05-10,NaT
166493,1174042,CARLOS,FURTADO,1973-01-29,NaT
166494,1174098,HOLLY,PILOTTE,1974-09-23,NaT


In [10]:
credit_score_data

,acctnbr,acctrolecd,acctroledesc,emplroleyn,persnbr,orgnbr,rundate,datelastmaint,firstname,lastname,datebirth,deathnotificationdate,taxid,Score_1,Prior Credit Score
23,103441,GUAR,Guarantor,N,1125397.0,NaN,2025-06-09,2025-06-09 21:22:51,MICHAEL,STEVENS,1979-09-18,NaT,36522609.0,790.0,790.0
24,103443,GUAR,Guarantor,N,1125397.0,NaN,2025-06-09,2025-06-09 21:22:51,MICHAEL,STEVENS,1979-09-18,NaT,36522609.0,790.0,790.0
42,103489,GUAR,Guarantor,N,1123682.0,NaN,2025-06-09,2025-06-09 21:22:51,RAYMOND,RENIERE,1962-12-14,NaT,38488979.0,758.0,758.0
77,103573,GUAR,Guarantor,N,1124298.0,NaN,2025-06-09,2025-06-09 21:22:51,STEPHEN,EGAN,1961-09-28,NaT,38461381.0,804.0,804.0
78,103573,GUAR,Guarantor,N,1124299.0,NaN,2025-06-09,2025-06-09 21:22:51,PAUL,SQUIZZERO,1963-12-15,NaT,39481632.0,742.0,742.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8016,151158287,GUAR,Guarantor,N,1063716.0,NaN,2025-06-09,2025-06-09 21:22:57,EDWARD,NASSER,1947-08-19,NaT,10383495.0,774.0,774.0
8019,151171560,GUAR,Guarantor,N,1123892.0,NaN,2025-06-09,2025-06-09 21:22:57,JASON,OUHRABKA,1973-01-26,NaT,36426726.0,714.0,714.0
8060,151189282,GUAR,Guarantor,N,1153879.0,NaN,2025-06-09,2025-06-09 21:22:57,DAVID,BERRYMAN,1974-12-02,NaT,17641758.0,722.0,722.0
8061,151189373,GUAR,Guarantor,N,1124930.0,NaN,2025-06-09,2025-06-09 21:22:57,RONALD,MEEHAN,1962-06-06,NaT,20589799.0,758.0,758.0


In [ ]:
# Control panel for additional parameters
loan_data = src.control_panel.criteria_flags(
    loan_data,
    credit_score_data,
    score_floor = 680, # Minimum Credit Score for any guarantor related to the loan
    score_decrease = -0.1, # Decrease of credit score since prior period
    ttm_pd_amt = 3, # Times in trailing 12 Months Past Due (15-29)
    ttm_pd30_amt = 1, # Times in trailing 12 Months Past Due (30+)
    ttm_overdrafts = 5, # Number of days with an overdrawn balance in trailing 12 Months
    deposit_change_pct = -.3, # Aggregate deposits in relationship decrease (-30% loss)
    min_deposits = 50000, # Minimum deposits to be eligible to flag deposit decrease
    utilization_limit = .7 # Line Utilization flag for trailing 12 months
)

# Consolidation of the columns necessary
final_df = loan_data[['acctnbr','effdate','ownername','product','loanofficer','inactivedate','Net Balance','Net Available','Net Collateral Reserve','cobal','creditlimitamt','Total Exposure_hhgroup','ttm_pd','ttm_pd30','TTM Days Overdrawn','3Mo_AvgBal','TTM_AvgBal','Deposit Change Pct','ttm line utilization','cleanup_provision','riskratingcd','past_due_flag','ttm_overdrafts_flag','deposit_change_flag','ttm_utilization_flag','score_flag','passed_all_flag']]

# Writing output
ALERTS_PATH = BASE_PATH / Path('./Output/alerts.xlsx')
final_df.to_excel(ALERTS_PATH, index=False, engine='openpyxl', sheet_name='Sheet1')

CREDSCORE_PATH = BASE_PATH / Path('./Output/credit_score_detail.xlsx')
credit_score_data.to_excel(CREDSCORE_PATH, index=False, engine='openpyxl', sheet_name='Sheet1')
    
print('Execution Complete!')

if __name__ == "__main__":
print(f"Starting alerts [{__version__}]")
# main(production_flag=True)
main()
print("Complete!")

    




